In [ ]:
import polars as pl
import sys
import os
import json

sys.path.append(os.path.dirname(os.path.abspath("")))

from libraries.client_stashapp import get_stashapp_client, StashAppClient
from libraries import browser

stash = get_stashapp_client()
stash_client = StashAppClient()

base_url = "https://stash.chiefsclub.com"

In [ ]:
# Browser functionality for injecting fix data
def check_browser_connection():
    """Check if Chrome is running with remote debugging enabled"""
    try:
        import requests
        response = requests.get('http://localhost:9222/json', timeout=5)
        if response.status_code == 200:
            pages = json.loads(response.text)
            print(f"✅ Chrome is running with remote debugging")
            print(f"📊 Found {len(pages)} open tabs/pages")
            return True
        else:
            print(f"❌ Chrome debugging port responded with status {response.status_code}")
            return False
    except Exception as e:
        print("❌ Cannot connect to Chrome debugging port")
        print("🔧 Please start Chrome with: --remote-debugging-port=9222")
        return False

def create_scenes_query_url(url_query):
    """Create a Stash query URL for scenes with Facial tag but excluding Cumshot tag"""
    import urllib.parse

    url = f"{base_url}/scenes?sortby=path&perPage=60"

    for component in url_query:
        # Convert to JSON string and use the specific URL encoding format that Stash expects
        query_json = json.dumps(component)
        # Replace curly braces with parentheses and square brackets with %5B %5D
        query_str = query_json.replace("{", "(").replace("}", ")").replace("[", "%5B").replace("]", "%5D")
        # URL encode the query string
        query_str = "&c=" + urllib.parse.quote(query_str, safe="():%5B%5D")
        url += query_str
    
    return url

def open_scenes_with_fix_data(check, check_matching_scenes):
    """Open the problematic scenes in browser with fix data injected"""
    
    if not check_matching_scenes:
        print("⚠️  No scenes found that need fixing")
        return False
    
    # Create URL for the scenes
    url = create_scenes_query_url(check["url_query"])
    
    print(f"🚀 Opening browser with {len(check_matching_scenes)} scenes that need to be fixed...")
    print(f"📋 Fix data will be injected as window.stashFixesData")
    print(f"🔗 URL: {url}")
    
    # Open the browser tab with fix data
    results = browser.open_or_update_tabs_with_data([url], check["fixes"], "stashFixesData")
    
    success = results.get(url, False)
    if success:
        print("✅ Successfully opened browser tab with fix data!")
    else:
        print("❌ Failed to open browser tab")
    
    return success

In [ ]:
checks = []

In [ ]:
name = "Facial without Cumshot"

stash_tags = stash_client.get_tags_by_names(["Facial", "Cumshot", "Fake Cum"])

modifier = "INCLUDES_ALL"

included_tags = [stash_tags.facial]
excluded_tags = [stash_tags.cumshot, stash_tags.fake_cum]

checks.append(
    {
        "name": name,
        "query": {
            "tags": {
                "value": [tag["id"] for tag in included_tags],
                "modifier": modifier,
                "excludes": [tag["id"] for tag in excluded_tags],
                "depth": 0,
            }
        },
        "fragment": "id title date tags { id name }",
        "url_query": [{
            "type": "tags",
            "modifier": modifier,
            "value": {
                "items": [
                    {"id": tag["id"], "label": tag["name"]}
                    for tag in included_tags
                ],
                "excluded": [
                    {
                        "id": tag["id"],
                        "label": tag["name"],
                    }
                    for tag in excluded_tags
                ],
                "depth": 0,
            },
        }],
        "fixes": [
            {
                "name": "Missing Cumshot Fix",
                "addTags": [
                    {
                        "id": stash_tags.cumshot["id"],
                        "label": stash_tags.cumshot["name"],
                    }
                ],
                "removeTags": [],
            }
        ],
    }
)

In [ ]:
name = "Both Straight and Lesbian without Multiple Sex Scenes In A Scene"

stash_tags = stash_client.get_tags_by_names(
    ["Twosome (Straight)", "Twosome (Lesbian)", "Multiple Sex Scenes In A Scene"]
)

modifier = "INCLUDES_ALL"

included_tags = [stash_tags.twosome_straight, stash_tags.twosome_lesbian]
excluded_tags = [stash_tags.multiple_sex_scenes_in_a_scene]

checks.append(
    {
        "name": name,
        "query": {
            "tags": {
                "value": [tag["id"] for tag in included_tags],
                "modifier": modifier,
                "excludes": [tag["id"] for tag in excluded_tags],
                "depth": 0,
            }
        },
        "fragment": "id title date tags { id name }",
        "url_query": [{
            "type": "tags",
            "modifier": modifier,
            "value": {
                "items": [
                    {"id": tag["id"], "label": tag["name"]} for tag in included_tags
                ],
                "excluded": [
                    {
                        "id": tag["id"],
                        "label": tag["name"],
                    }
                    for tag in excluded_tags
                ],
                "depth": 0,
            },
        }],
        "fixes": [
            {
                "name": "Remove Twosome (Straight)",
                "addTags": [],
                "removeTags": [
                    {
                        "id": stash_tags.twosome_straight["id"],
                        "name": stash_tags.twosome_straight["name"],
                    },
                ],
            },
            {
                "name": "Remove Twosome (Lesbian)",
                "addTags": [],
                "removeTags": [
                    {
                        "id": stash_tags.twosome_lesbian["id"],
                        "name": stash_tags.twosome_lesbian["name"],
                    },
                ],
            },
        ],
    }
)

In [ ]:
name = "Both BBG and BGG without Multiple Sex Scenes In A Scene"

stash_tags = stash_client.get_tags_by_names(
    ["Threesome (BBG)", "Threesome (BGG)", "Multiple Sex Scenes In A Scene"]
)

modifier = "INCLUDES_ALL"

included_tags = [stash_tags.threesome_bbg, stash_tags.threesome_bgg]
excluded_tags = [stash_tags.multiple_sex_scenes_in_a_scene]

checks.append({
    "name": name,
    "query": {
        "tags": {
            "value": [tag["id"] for tag in included_tags],
            "modifier": modifier,
            "excludes": [tag["id"] for tag in excluded_tags],
            "depth": 0,
        }
    },
    "fragment": "id title date tags { id name }",
    "url_query": [{
        "type": "tags",
        "modifier": modifier,
        "value": {
            "items": [
                {"id": tag["id"], "label": tag["name"]}
                for tag in included_tags
            ],
            "excluded": [
                {"id": tag["id"], "label": tag["name"]}
                for tag in excluded_tags
            ],
            "depth": 0,
        },
    }],
    "fixes": [
        {
            "name": "Remove Threesome (BBG)",
            "addTags": [],
            "removeTags": [
                {
                    "id": stash_tags.threesome_bbg["id"],
                    "name": stash_tags.threesome_bbg["name"],
                },
            ],
        },
        {
            "name": "Remove Threesome (BGG)",
            "addTags": [],
            "removeTags": [
                {
                    "id": stash_tags.threesome_bgg["id"],
                    "name": stash_tags.threesome_bgg["name"],
                },
            ],
        },
    ],
})

In [ ]:
name = "Data Quality: 2 Performers No Group Makeup, Tags Indicate Straight"

stash_tags = stash_client.get_tags_by_names(
    [
        "Blowjob",
        "Facial",
        "Futanari",
        "Cumshot",
        "Behind the Scenes",
        "Full Movie",
        "Category: Group Makeup",
        "Twosome",
        "Twosome (Straight)",
        "Group Makeup Verified",
        "Transgender Female",
    ]
)

modifier = "INCLUDES"
depth = -1

included_tags = [stash_tags.blowjob, stash_tags.facial, stash_tags.cumshot]
excluded_tags = [
    stash_tags.behind_the_scenes,
    stash_tags.full_movie,
    stash_tags.category_group_makeup,
    stash_tags.futanari,
]

checks.append(
    {
        "name": name,
        "query": {
            "performer_count": {"modifier": "EQUALS", "value": 2},
            "tags": {
                "value": [tag["id"] for tag in included_tags],
                "modifier": modifier,
                "excludes": [tag["id"] for tag in excluded_tags],
                "depth": depth,
            },
            "performer_tags": {
                "value": [],
                "modifier": modifier,
                "excludes": [stash_tags.transgender_female["id"]],
                "depth": depth,
            },
        },
        "fragment": "id title date tags { id name }",
        "url_query": [
            {"type": "performer_count", "modifier": "EQUALS", "value": 2},
            {
                "type": "tags",
                "modifier": modifier,
                "value": {
                    "items": [
                        {"id": tag["id"], "label": tag["name"]} for tag in included_tags
                    ],
                    "excluded": [
                        {"id": tag["id"], "label": tag["name"]} for tag in excluded_tags
                    ],
                    "depth": depth,
                },
            },
            {
                "type": "performer_tags",
                "modifier": "INCLUDES_ALL",
                "value": {
                    "items": [],
                    "excluded": [{"id": stash_tags.transgender_female["id"], "label": stash_tags.transgender_female["name"]}],
                    "depth": 0,
                }
            }
        ],
        "fixes": [
            {
                "name": "Add Group Makeup Verified, Twosome, Twosome (Straight)",
                "addTags": [
                    {
                        "id": stash_tags.group_makeup_verified["id"],
                        "name": stash_tags.group_makeup_verified["name"],
                    },
                    {
                        "id": stash_tags.twosome["id"],
                        "name": stash_tags.twosome["name"],
                    },
                    {
                        "id": stash_tags.twosome_straight["id"],
                        "name": stash_tags.twosome_straight["name"],
                    },
                ],
            }
        ],
    }
)

In [ ]:
name = "Data Quality: Futanari XXX without Futanari tag"

stash_tags = stash_client.get_tags_by_names(["Futanari"])

futanari_studio = stash.find_studio("Futanari XXX")

modifier = "INCLUDES"
depth = 0

included_tags = []
excluded_tags = [stash_tags.futanari]

checks.append(
    {
        "name": name,
        "query": {
            "tags": {
                "value": [tag["id"] for tag in included_tags],
                "modifier": modifier,
                "excludes": [tag["id"] for tag in excluded_tags],
                "depth": depth,
            },
            "studios": {
                "modifier": "INCLUDES",
                "value": [futanari_studio["id"]],
                "depth": 0,
            },
        },
        "fragment": "id title date tags { id name }",
        "url_query": [
            {
                "type": "tags",
                "modifier": modifier,
                "value": {
                    "items": [
                        {"id": tag["id"], "label": tag["name"]} for tag in included_tags
                    ],
                    "excluded": [
                        {"id": tag["id"], "label": tag["name"]} for tag in excluded_tags
                    ],
                    "depth": depth,
                },
            },
            {
                "type": "studios",
                "modifier": "INCLUDES",
                "value": {
                    "items": [
                        {"id": futanari_studio["id"], "label": futanari_studio["name"]}
                    ],
                    "excluded": [],
                    "depth": 0,
                },
            },
        ],
        "fixes": [
            {
                "name": "Add Futanari XXX",
                "addTags": [
                    {
                        "id": stash_tags.futanari["id"],
                        "name": stash_tags.futanari["name"],
                    },
                ],
            }
        ],
    }
)

In [ ]:
checks_with_matching_scenes = []

for check in checks:
    matching_scenes = stash.find_scenes(check["query"], fragment=check["fragment"])
    print(f"Found {len(matching_scenes)} scenes for {check['name']}")
    if len(matching_scenes) > 0:
        checks_with_matching_scenes.append(check)

In [ ]:
# Check browser connection
browser_ready = check_browser_connection()

current_check = checks_with_matching_scenes[-1]
current_matching_scenes = stash.find_scenes(current_check["query"], fragment=current_check["fragment"])

if not browser_ready:
    print("⚠️  Browser not ready. Start Chrome with:")
    print('Start-Process "chrome.exe" -ArgumentList "--remote-debugging-port=9222", "--user-data-dir=C:\\\\temp\\\\chrome-debug"')
else:
    open_scenes_with_fix_data(current_check, current_matching_scenes)